In [1]:
import polars
from xlsxwriter import Workbook

In [2]:
publications = polars.read_database_uri(
    """SELECT publication_id, doi, rank_sum
    FROM work_publications
    WHERE rank_sum IS NOT NULL
    ORDER BY rank_sum""",
    "sqlite://sydney.sqlite"
)
publications

publication_id,doi,rank_sum
i64,str,i64
62,"""10.1016/j.jii.2020.100125""",0
70,"""10.1056/NEJMra1906193""",0
85,"""10.1109/TCSI.2020.3036847""",0
138,"""10.1111/nph.18016""",0
152,"""10.1016/j.anr.2019.11.002""",0
…,…,…
543,"""10.1038/s41591-021-01498-0""",249998
2926,"""10.1093/database/baz085""",249998
50,"""10.1016/S0140-6736(22)01438-6""",249999


In [3]:
authors = polars.read_database_uri(
    """SELECT a.external_author_id, author_names, COUNT(*) AS publication_count,
        group_concat(distinct qs_subject_area_field_name) as qs_subjects
    FROM work_publication_authors pa
    JOIN work_external_authors a ON pa.external_author_id = a.external_author_id
    JOIN work_publications p ON pa.publication_id = p.publication_id
    GROUP BY a.external_author_id
    """,
    "sqlite://sydney.sqlite"
)
authors

external_author_id,author_names,publication_count,qs_subjects
i64,str,i64,str
1,"""Troon, S.""",1,"""Life Sciences & Medicine"""
2,"""Simoniello, C.""",1,"""Natural Sciences| Engineering …"
3,"""Beerman, S.B.""",1,"""Life Sciences & Medicine"""
4,"""Sariyildiz, E.""",1,"""Life Sciences & Medicine| Engi…"
5,"""Anazodo, A.| Anazodo, A.C.""",5,"""Life Sciences & Medicine,Arts …"
…,…,…,…
61175,"""Zhu, X.""",1,"""Natural Sciences"""
61176,"""McIntyre, D.""",1,"""Life Sciences & Medicine"""
61177,"""Boyle, J.""",1,"""Life Sciences & Medicine"""


In [4]:
with Workbook("sydney.xlsx") as wb:
    publications.write_excel(wb, "publications", autofit=True)
    authors.write_excel(wb, "authors", autofit=True)